<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 2 / SPRINT 3 / NOTE 1*

# 📝 Assignment
---
# Choose your ML problems

이번 스프린트에서는 포트폴리오 프로젝트를 위해 수강생 각자가 정한 데이터셋을 사용하여 과제를 수행합니다. 프로젝트를 위한 데이터를 최종 선택하기 전에 여러 가능한 데이터세트를 살펴보고 간단한 모델까지 학습해 보는 것을 추천드립니다.

### 1) 타겟으로 사용할 특성을 선택합니다.
**(Urclass Quiz)** 여러분들이 프로젝트에 사용할 데이터셋과 타겟특성에 대해 아래의 내용을 포함하여 간략하게 설명해주세요. 
* `df.dtypes`을 사용해서 타겟특성의 데이터 타입 확인
* `df.describe()`를 사용해서 타겟특성의 대표값 확인(실수-평균, 범주형-최빈값)

In [1]:
from google.colab import drive
drive.mount('/content/mydrive')

Drive already mounted at /content/mydrive; to attempt to forcibly remount, call drive.mount("/content/mydrive", force_remount=True).


In [4]:
# 성인 인구 소득 예측
# target = ['income']
import pandas as pd

df = pd.read_csv('/content/mydrive/MyDrive/AIB/Section2/adult.csv')
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [6]:
df.dtypes
# numerical feature : age, fnlwgt, education.num, capital.gain, capitl.loss, hours.per.week
# categorical feature : workclass, education, marital.status, occupation, relationship, race, sex, native.country

# target인 income은 categorcial이므로 분류 문제로 생각하고 모델을 설계

age                int64
workclass         object
fnlwgt             int64
education         object
education.num      int64
marital.status    object
occupation        object
relationship      object
race              object
sex               object
capital.gain       int64
capital.loss       int64
hours.per.week     int64
native.country    object
income            object
dtype: object

In [10]:
df['income'].unique()

array(['<=50K', '>50K'], dtype=object)

In [11]:
df.describe(exclude = 'number')['income']

# income은 '<=50K', '>50K' 두가지 category가 있으며 그 중 최빈값은 '<=50K'이다.

count     32561
unique        2
top       <=50K
freq      24720
Name: income, dtype: object

### 2) 풀고자 하는 문제가 분류/회귀 문제인지 정합니다.
**(Urclass Quiz)** 다이아몬드를 보관하기 위해 금고를 만들었습니다. 금고가 너무 작아서 가격이 상위 25%인 것만 보관하려고 합니다. 이를 위해 'price' 특성을 이용해 금고에 보관할 다이아몬드와 그렇지 않은 것으로 분류하는 이진분류 컬럼을 만들어 타겟 특성으로 사용할 것입니다. 타겟 특성의 최빈값의 빈도수를 적어주세요.
- 'top25'라는 새로운 타겟변수(컬럼)를 만들어주세요.
- 상위 25% 이상 True, 상위 25% 미만 False

   

In [12]:
import seaborn as sns
diamonds = sns.load_dataset('diamonds')

In [18]:
import numpy as np

top = np.percentile(diamonds['price'], 75)
diamonds['top25'] = np.where(diamonds['price'] >= top, True, False)

diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z,top25
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,False
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,False
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31,False
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63,False
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75,False


In [21]:
print('타겟 특성의 최빈값의 빈도수 =', diamonds['top25'].value_counts()[0])

타겟 특성의 최빈값의 빈도수 = 40455


### 3) 타겟의 분포를 분석하고 사용할 평가지표를 선택합니다. 평가지표를 선택한 이유를 설명하세요.
- 분류: 클래스가 몇 개인지? 클래스 비율이 어떤지(balance or imbalance)
- 회귀: 타겟 분포가 right-skewed 인지? log-transform을 사용할 것인지?
- 분류: 다수 클래스(majority class) 비율이 50%~70% 인 경우 정확도(accuracy)만 사용해도 무방합니다. 하지만 범위를 넘어설 경우 정확도만을 사용해서는 모델을 잘못 이해할 수 있습니다. 어떤 평가지표를 사용해야 할까요?
- 회귀: MAE, RMSE, R^2 등을 사용할 수 있습니다.

---


**(Urclass Quiz)** 여러분들이 프로젝트에 사용할 데이터의 기준모델과 사용할 평가지표를 제출해주세요.

In [22]:
# 클래스의 비율을 확인
df['income'].value_counts()/len(df)

# 기준 모델로는 최빈값을 이용
# 평가 지표로는 '<=50K'가 75% 이상을 차지하므로 accuracy 사용은 부적합, f1-score 이용

<=50K    0.75919
>50K     0.24081
Name: income, dtype: float64

## 🔥 도전과제(Github - Discussion)

### 4) 실무에서 데이터분석가/데이터엔지니어/데이터사이언티스트에게 중요한 역량이 무엇이라고 생각하며, 어떤 어려움을 겪게 될 것 같은지와 어떤 능력이 필요할 지 논의해 보세요. 또한, 섹션2 프로젝트에 앞서 본인이 얻고자 하는 점이나 다짐 등을 서로 공유해 보세요. 


In [ ]:
### 이곳에서 과제를 진행해 주세요 ### 